In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D, UpSampling2D, AveragePooling2D,ZeroPadding2D,Convolution2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
import tensorflow.keras.backend as K
from IPython.display import SVG, Image
import cv2
import utils

In [4]:
img_size=(100,100)
batch_size = 32
datagen_train=ImageDataGenerator(horizontal_flip=True)
train_generator=datagen_train.flow_from_directory("EyeTrack_Augmented/training/EyeTrack_Augmented",
target_size=img_size,
batch_size=batch_size,
class_mode='categorical',
color_mode ="grayscale",
shuffle=True)

datagen_validation=ImageDataGenerator(horizontal_flip=True)
validation_generator=datagen_validation.flow_from_directory("EyeTrack_Augmented/validation/EyeTrack_Augmented",
target_size=img_size,
batch_size=batch_size,
class_mode='categorical',
color_mode ="grayscale",
shuffle=True)

datagen_test=ImageDataGenerator(horizontal_flip=True)
test_generator=datagen_test.flow_from_directory("EyeTrack_Augmented/testing/EyeTrack_Augmented",
target_size=img_size,
batch_size=batch_size,
class_mode='categorical',
color_mode ="grayscale",
shuffle=True)

Found 2785 images belonging to 2 classes.
Found 347 images belonging to 2 classes.
Found 348 images belonging to 2 classes.


# InceptionV3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [24]:

inception = InceptionV3(input_shape=[100,100]+ [1], weights=None, include_top=False)

In [25]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [26]:
x = Flatten()(inception.output)

In [27]:
prediction = Dense(2, activation='sigmoid')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [28]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 49, 49, 32)   288         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 49, 49, 32)   96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 49, 49, 32)   0           batch_normalization_282[0][0]    
____________________________________________________________________________________________

In [29]:
def precision(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
optimum= Adam(learning_rate=0.000005)
model.compile(loss='binary_crossentropy', optimizer=optimum, metrics=['accuracy',recall,precision,tf.keras.metrics.AUC(curve="ROC"),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.SensitivityAtSpecificity(0.5)])

In [30]:
%%time
batch_size = 32


ephocs=35
                    

steps_per_epoch=train_generator.n//train_generator.batch_size
steps_per_epoch
validation_steps=validation_generator.n//validation_generator.batch_size
validation_steps
mod = model.fit(x=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=ephocs,
                    validation_data=validation_generator,
                    validation_steps=validation_steps
                    )

Epoch 1/35
87/87 [==============================] - 20s 125ms/step - loss: 0.6933 - accuracy: 0.6012 - recall: 0.6056 - precision: 0.6056 - auc_2: 0.3747 - false_positives_2: 1098.0000 - true_positives_2: 1655.0000 - false_negatives_2: 1098.0000 - true_negatives_2: 1655.0000 - sensitivity_at_specificity_2: 0.0425 - val_loss: 0.6937 - val_accuracy: 0.5562 - val_recall: 0.5562 - val_precision: 0.5562 - val_auc_2: 0.3610 - val_false_positives_2: 142.0000 - val_true_positives_2: 178.0000 - val_false_negatives_2: 142.0000 - val_true_negatives_2: 178.0000 - val_sensitivity_at_specificity_2: 0.0594
Epoch 2/35
87/87 [==============================] - 8s 92ms/step - loss: 0.6932 - accuracy: 0.6008 - recall: 0.6052 - precision: 0.6052 - auc_2: 0.3817 - false_positives_2: 1099.0000 - true_positives_2: 1654.0000 - false_negatives_2: 1099.0000 - true_negatives_2: 1654.0000 - sensitivity_at_specificity_2: 0.0559 - val_loss: 0.6939 - val_accuracy: 0.5344 - val_recall: 0.5344 - val_precision: 0.5344 -

In [31]:
model.evaluate(test_generator,verbose=1)

11/11 [==============================] - 2s 154ms/step - loss: 0.6906 - accuracy: 0.6868 - recall: 0.6445 - precision: 0.7256 - auc_2: 0.7909 - false_positives_2: 86.0000 - true_positives_2: 224.0000 - false_negatives_2: 124.0000 - true_negatives_2: 262.0000 - sensitivity_at_specificity_2: 0.3218


[0.690595805644989,
 0.6867815852165222,
 0.6444805264472961,
 0.7256469130516052,
 0.7908574342727661,
 86.0,
 224.0,
 124.0,
 262.0,
 0.3218390941619873]

# Resnet50

In [45]:
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input

In [46]:
resnet = ResNet50(input_shape=[100,100]+ [1], weights=None, include_top=False)

In [47]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [48]:
x = Flatten()(resnet.output)

In [49]:
prediction = Dense(2, activation='sigmoid')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [50]:
# view the structure of the model
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 1)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 50, 50, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 50, 50, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [51]:
def precision(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
optimum= Adam(learning_rate=0.000005)
model.compile(loss='binary_crossentropy', optimizer=optimum, metrics=['accuracy',recall,precision,tf.keras.metrics.AUC(curve="ROC"),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.SensitivityAtSpecificity(0.5)])

In [52]:
%%time
batch_size = 32


ephocs=35
                    

steps_per_epoch=train_generator.n//train_generator.batch_size
steps_per_epoch
validation_steps=validation_generator.n//validation_generator.batch_size
validation_steps
mod = model.fit(x=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=ephocs,
                    validation_data=validation_generator,
                    validation_steps=validation_steps
                    )

Epoch 1/35
87/87 [==============================] - 12s 105ms/step - loss: 0.6369 - accuracy: 0.5449 - recall: 0.5172 - precision: 0.5915 - auc_5: 0.6597 - false_positives_5: 995.0000 - true_positives_5: 1409.0000 - false_negatives_5: 1344.0000 - true_negatives_5: 1758.0000 - sensitivity_at_specificity_5: 0.5826 - val_loss: 0.5947 - val_accuracy: 0.6438 - val_recall: 0.5875 - val_precision: 0.6747 - val_auc_5: 0.7631 - val_false_positives_5: 91.0000 - val_true_positives_5: 188.0000 - val_false_negatives_5: 132.0000 - val_true_negatives_5: 229.0000 - val_sensitivity_at_specificity_5: 0.7719
Epoch 2/35
87/87 [==============================] - 8s 90ms/step - loss: 0.5885 - accuracy: 0.7827 - recall: 0.7062 - precision: 0.7920 - auc_5: 0.8534 - false_positives_5: 514.0000 - true_positives_5: 1966.0000 - false_negatives_5: 818.0000 - true_negatives_5: 2270.0000 - sensitivity_at_specificity_5: 0.9192 - val_loss: 0.5604 - val_accuracy: 0.9000 - val_recall: 0.8719 - val_precision: 0.8904 - val

In [53]:
model.evaluate(test_generator,verbose=1)

11/11 [==============================] - 1s 113ms/step - loss: 0.2207 - accuracy: 0.9425 - recall: 0.9412 - precision: 0.9412 - auc_5: 0.9847 - false_positives_5: 20.0000 - true_positives_5: 328.0000 - false_negatives_5: 20.0000 - true_negatives_5: 328.0000 - sensitivity_at_specificity_5: 0.9971


[0.22066135704517365,
 0.9425287246704102,
 0.9411525726318359,
 0.9411525726318359,
 0.9847114682197571,
 20.0,
 328.0,
 20.0,
 328.0,
 0.9971264600753784]

# VGG16

In [54]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

In [55]:
vgg16 = VGG16(input_shape=[100,100]+ [1], weights=None, include_top=False)

In [56]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [57]:
x = Flatten()(vgg16.output)

In [58]:
prediction = Dense(2, activation='sigmoid')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [59]:
# view the structure of the model
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 100, 100, 1)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0   

In [60]:
def precision(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
optimum= Adam(learning_rate=0.000005)
model.compile(loss='binary_crossentropy', optimizer=optimum, metrics=['accuracy',recall,precision,tf.keras.metrics.AUC(curve="ROC"),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.SensitivityAtSpecificity(0.5)])

In [61]:
%%time
batch_size = 32


ephocs=35
                    

steps_per_epoch=train_generator.n//train_generator.batch_size
steps_per_epoch
validation_steps=validation_generator.n//validation_generator.batch_size
validation_steps
mod = model.fit(x=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=ephocs,
                    validation_data=validation_generator,
                    validation_steps=validation_steps
                    )

Epoch 1/35
87/87 [==============================] - 10s 97ms/step - loss: 0.6938 - accuracy: 0.6001 - recall: 0.1667 - precision: 0.4717 - auc_6: 0.5239 - false_positives_6: 521.0000 - true_positives_6: 464.0000 - false_negatives_6: 2289.0000 - true_negatives_6: 2232.0000 - sensitivity_at_specificity_6: 0.5376 - val_loss: 0.6944 - val_accuracy: 0.5437 - val_recall: 0.1437 - val_precision: 0.4717 - val_auc_6: 0.4882 - val_false_positives_6: 59.0000 - val_true_positives_6: 46.0000 - val_false_negatives_6: 274.0000 - val_true_negatives_6: 261.0000 - val_sensitivity_at_specificity_6: 0.4906
Epoch 2/35
87/87 [==============================] - 8s 90ms/step - loss: 0.6928 - accuracy: 0.6044 - recall: 0.1685 - precision: 0.5663 - auc_6: 0.5646 - false_positives_6: 354.0000 - true_positives_6: 469.0000 - false_negatives_6: 2284.0000 - true_negatives_6: 2399.0000 - sensitivity_at_specificity_6: 0.5369 - val_loss: 0.6930 - val_accuracy: 0.5750 - val_recall: 0.1250 - val_precision: 0.5272 - val_au

In [62]:
model.evaluate(test_generator,verbose=1)

11/11 [==============================] - 1s 136ms/step - loss: 0.6695 - accuracy: 0.8247 - recall: 0.8024 - precision: 0.8418 - auc_6: 0.9009 - false_positives_6: 53.0000 - true_positives_6: 279.0000 - false_negatives_6: 69.0000 - true_negatives_6: 295.0000 - sensitivity_at_specificity_6: 0.9684


[0.6695393323898315,
 0.8247126340866089,
 0.8023538589477539,
 0.8417854309082031,
 0.9008950591087341,
 53.0,
 279.0,
 69.0,
 295.0,
 0.9683908224105835]

# VGG19

In [63]:
vgg19 = VGG19(input_shape=[100,100]+ [1], weights=None, include_top=False)

In [64]:
# don't train existing weights
for layer in vgg19.layers:
    layer.trainable = False

In [65]:
x = Flatten()(vgg19.output)

In [66]:
prediction = Dense(2, activation='sigmoid')(x)

# create a model object
model = Model(inputs=vgg19.input, outputs=prediction)

In [67]:
# view the structure of the model
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 100, 100, 1)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0   

In [68]:
def precision(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
optimum= Adam(learning_rate=0.000005)
model.compile(loss='binary_crossentropy', optimizer=optimum, metrics=['accuracy',recall,precision,tf.keras.metrics.AUC(curve="ROC"),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.SensitivityAtSpecificity(0.5)])

In [69]:
%%time
batch_size = 32


ephocs=35
                    

steps_per_epoch=train_generator.n//train_generator.batch_size
steps_per_epoch
validation_steps=validation_generator.n//validation_generator.batch_size
validation_steps
mod = model.fit(x=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=ephocs,
                    validation_data=validation_generator,
                    validation_steps=validation_steps
                    )

Epoch 1/35
87/87 [==============================] - 9s 96ms/step - loss: 0.6932 - accuracy: 0.6033 - recall: 0.7640 - precision: 0.6058 - auc_7: 0.4499 - false_positives_7: 1351.0000 - true_positives_7: 2127.0000 - false_negatives_7: 626.0000 - true_negatives_7: 1402.0000 - sensitivity_at_specificity_7: 0.2274 - val_loss: 0.6934 - val_accuracy: 0.5531 - val_recall: 0.8188 - val_precision: 0.5561 - val_auc_7: 0.4191 - val_false_positives_7: 209.0000 - val_true_positives_7: 262.0000 - val_false_negatives_7: 58.0000 - val_true_negatives_7: 111.0000 - val_sensitivity_at_specificity_7: 0.1594
Epoch 2/35
87/87 [==============================] - 8s 89ms/step - loss: 0.6929 - accuracy: 0.6306 - recall: 0.9080 - precision: 0.5914 - auc_7: 0.4500 - false_positives_7: 1770.0000 - true_positives_7: 2497.0000 - false_negatives_7: 256.0000 - true_negatives_7: 983.0000 - sensitivity_at_specificity_7: 0.1435 - val_loss: 0.6930 - val_accuracy: 0.5844 - val_recall: 0.9344 - val_precision: 0.5532 - val_a

In [70]:
model.evaluate(test_generator,verbose=1)

11/11 [==============================] - 1s 78ms/step - loss: 0.6841 - accuracy: 0.4138 - recall: 0.4205 - precision: 0.4180 - auc_7: 0.6408 - false_positives_7: 204.0000 - true_positives_7: 146.0000 - false_negatives_7: 202.0000 - true_negatives_7: 144.0000 - sensitivity_at_specificity_7: 0.4023


[0.6841331720352173,
 0.4137931168079376,
 0.4204545319080353,
 0.4180440902709961,
 0.6408458948135376,
 204.0,
 146.0,
 202.0,
 144.0,
 0.40229883790016174]